In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
def train_linear_regression_model(df, include_ratios=False, cv_folds=5):
    """
    Trains a Linear Regression model on censored=0 data to predict final_expandEvts and validates on all data
    to assess generalizability. Uses relative error metrics (sMAPE, sMdAPE, Normalized RMSE) and cross-validation.

    Parameters:
    - df (pandas.DataFrame): Preprocessed DataFrame with solver features, ratio features, and final_expandEvts.
    - include_ratios (bool): If True, includes ratio features; if False, excludes them (default: False).
    - cv_folds (int): Number of cross-validation folds (default: 5).

    Returns:
    - dict: Contains relative error metrics, CV scores, and feature coefficients for censored=0 test set and all data.
    """
    try:
        # Validate input
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"Input 'df' must be a pandas DataFrame, got {type(df)}")
        print(f"Input DataFrame type: {type(df)}")
        
        # Feature selection
        exclude_cols = ['filename', 'final_expandEvts', 'stop_iter', 'final_maxStackDepth']
        if not include_ratios:
            exclude_cols.extend([f'expandEvts_ratio_{i}' for i in range(1, 6)])
            exclude_cols.extend([f'pruneBacktrackEvts_ratio_{i}' for i in range(1, 6)])
        features = [col for col in df.columns if col not in exclude_cols]
        
        # Print target statistics
        print("\nTarget Statistics (final_expandEvts):")
        mean_censored = df[df['censored'] == 0]['final_expandEvts'].mean()
        std_censored = df[df['censored'] == 0]['final_expandEvts'].std()
        mean_all = df['final_expandEvts'].mean()
        std_all = df['final_expandEvts'].std()
        print(f"Mean (censored=0): {mean_censored:.2f}")
        print(f"Std Dev (censored=0): {std_censored:.2f}")
        print(f"Mean (all data): {mean_all:.2f}")
        print(f"Std Dev (all data): {std_all:.2f}")
        
        # Calculate relative error metrics
        def calculate_smape(y_true, y_pred):
            """Calculate Symmetric Mean Absolute Percentage Error (sMAPE)."""
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            denominator = np.abs(y_true) + np.abs(y_pred)
            mask = denominator != 0  # Avoid division by zero
            if mask.sum() == 0:
                return np.nan
            smape = np.mean(2 * np.abs(y_true[mask] - y_pred[mask]) / denominator[mask]) * 100
            return smape
        
        def calculate_smdape(y_true, y_pred):
            """Calculate Symmetric Median Absolute Percentage Error (sMdAPE)."""
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            denominator = np.abs(y_true) + np.abs(y_pred)
            mask = denominator != 0
            if mask.sum() == 0:
                return np.nan
            smape_terms = 2 * np.abs(y_true[mask] - y_pred[mask]) / denominator[mask] * 100
            return np.median(smape_terms)
        
        def calculate_normalized_rmse(y_true, y_pred, mean_y):
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            return rmse / mean_y if mean_y != 0 else np.nan
        
        # Train on censored=0 data
        print("\nTraining Linear Regression on censored=0 data...")
        df_censored = df[df['censored'] == 0]
        if df_censored.empty:
            print("Warning: No censored=0 instances found. Exiting.")
            return None
        
        X_censored = df_censored[features]
        y_censored = df_censored['final_expandEvts']
        X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_censored, y_censored, test_size=0.2, random_state=42)
        
        lr_model = LinearRegression()
        lr_model.fit(X_train_c, y_train_c)
        
        # Evaluate on censored=0 test set
        y_pred_c = lr_model.predict(X_test_c)
        rmse_censored = np.sqrt(mean_squared_error(y_test_c, y_pred_c))
        norm_rmse_censored = calculate_normalized_rmse(y_test_c, y_pred_c, mean_censored)
        smape_censored = calculate_smape(y_test_c, y_pred_c)
        smdape_censored = calculate_smdape(y_test_c, y_pred_c)
        cv_scores = cross_val_score(lr_model, X_censored, y_censored, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_norm_rmse_censored = (-cv_scores.mean()) / mean_censored if mean_censored != 0 else np.nan
        print(f"Censored=0 Test Set RMSE: {rmse_censored:.4f}")
        print(f"Censored=0 Test Set Normalized RMSE: {norm_rmse_censored:.4%}")
        print(f"Censored=0 Test Set sMAPE: {smape_censored:.2f}%")
        print(f"Censored=0 Test Set sMdAPE: {smdape_censored:.2f}%")
        print(f"Censored=0 CV Normalized RMSE: {cv_norm_rmse_censored:.4%}")
        
        # Feature coefficients
        coefficients = pd.DataFrame({
            'feature': X_censored.columns,
            'coefficient': lr_model.coef_
        }).sort_values('coefficient', key=abs, ascending=False)
        print("\nCensored=0 Feature Coefficients (Top 10):")
        print(coefficients.head(10))
        
        # Validate on all data
        print("\nValidating on all data...")
        X_all = df[features]
        y_all = df['final_expandEvts']
        y_pred_all = lr_model.predict(X_all)
        rmse_all = np.sqrt(mean_squared_error(y_all, y_pred_all))
        norm_rmse_all = calculate_normalized_rmse(y_all, y_pred_all, mean_all)
        smape_all = calculate_smape(y_all, y_pred_all)
        smdape_all = calculate_smdape(y_all, y_pred_all)
        print(f"All Data RMSE: {rmse_all:.4f}")
        print(f"All Data Normalized RMSE: {norm_rmse_all:.4%}")
        print(f"All Data sMAPE: {smape_all:.2f}%")
        print(f"All Data sMdAPE: {smdape_all:.2f}%")

        # Cross-validation for all data
        print("\nPerforming cross-validation on all data...")
        cv_scores_all = cross_val_score(lr_model, X_all, y_all, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_norm_rmse_all = (-cv_scores_all.mean()) / mean_all if mean_all != 0 else np.nan
        print(f"All Data CV Normalized RMSE: {cv_norm_rmse_all:.4%}")
        
        return {
            'rmse_censored_test': rmse_censored,
            'norm_rmse_censored_test': norm_rmse_censored,
            'smape_censored_test': smape_censored,
            'smdape_censored_test': smdape_censored,
            'cv_norm_rmse_censored': cv_norm_rmse_censored,
            'rmse_all': rmse_all,
            'norm_rmse_all': norm_rmse_all,
            'smape_all': smape_all,
            'smdape_all': smdape_all,
            'cv_norm_rmse_all': cv_norm_rmse_all,
            'feature_coefficients': coefficients,
        }
    
    except Exception as e:
        print(f"Error in train_linear_regression_model: {e}")
        return None

df = pd.read_excel("structured_data.xlsx")
results = train_linear_regression_model(df, include_ratios=True)
results

Input DataFrame type: <class 'pandas.core.frame.DataFrame'>

Target Statistics (final_expandEvts):
Mean (censored=0): 53227800.02
Std Dev (censored=0): 174281183.67
Mean (all data): 761155886.29
Std Dev (all data): 636531467.58

Training Linear Regression on censored=0 data...
Censored=0 Test Set RMSE: 5536596.9412
Censored=0 Test Set Normalized RMSE: 10.4017%
Censored=0 Test Set sMAPE: 118.29%
Censored=0 Test Set sMdAPE: 190.38%
Censored=0 CV Normalized RMSE: 84.0312%

Censored=0 Feature Coefficients (Top 10):
                       feature   coefficient
58  pruneBacktrackEvts_ratio_5  4.500800e+07
52  pruneBacktrackEvts_ratio_2 -2.835557e+07
56  pruneBacktrackEvts_ratio_4 -2.368744e+07
22              avg_expandEvts -2.303088e+07
54  pruneBacktrackEvts_ratio_3  2.139240e+07
51          expandEvts_ratio_2 -2.096127e+07
57          expandEvts_ratio_5 -1.868529e+07
53          expandEvts_ratio_3  1.218121e+07
20                    avg_evts  1.092435e+07
55          expandEvts_ratio_4 -9

{'rmse_censored_test': 5536596.941187796,
 'norm_rmse_censored_test': 0.10401701627256087,
 'smape_censored_test': 118.28975606293271,
 'smdape_censored_test': 190.3783445151774,
 'cv_norm_rmse_censored': 0.840312314399647,
 'rmse_all': 568905739983.9944,
 'norm_rmse_all': 747.4234256465876,
 'smape_all': 165.71537167294437,
 'smdape_all': 200.0,
 'cv_norm_rmse_all': 8.02944097786046,
 'feature_coefficients':                        feature   coefficient
 58  pruneBacktrackEvts_ratio_5  4.500800e+07
 52  pruneBacktrackEvts_ratio_2 -2.835557e+07
 56  pruneBacktrackEvts_ratio_4 -2.368744e+07
 22              avg_expandEvts -2.303088e+07
 54  pruneBacktrackEvts_ratio_3  2.139240e+07
 ..                         ...           ...
 4              backtrackEvts_1 -2.738627e-02
 6              maxStackDepth_1  1.342053e-02
 19                    censored -9.122305e-07
 49          expandEvts_ratio_1  0.000000e+00
 50  pruneBacktrackEvts_ratio_1  0.000000e+00
 
 [71 rows x 2 columns]}